# Modellaufbau und Training

---

Autor: mn086

---

## Setup

In [1]:
import os
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import altair as alt

## Daten-Import

**Pfade:**

In [2]:
root_processed = os.path.join('..', 'data', 'processed')

**Import in Dataframes**

In [3]:
df_regr = pd.read_csv(os.path.join(root_processed, 'regression_data.csv'))

## Daten-Struktur

In [4]:
df_regr.tail(3)

,anzahl_personen,vee,anzahl_kfz_je_person,unfaelle_je_10k_kfz,elektro,pih,euro2,euro3,euro4,euro6,euro6dt
396,NaN,25966,NaN,50.5,0.246801,0.123908,5.708917,6.968312,27.723949,23.002234,4.853748
397,NaN,26021,NaN,51.7,0.279993,0.168433,6.172961,7.648391,27.577080,22.237534,5.418294
398,NaN,25954,NaN,63.3,0.344202,0.192435,4.607541,6.389057,24.128833,26.149405,6.292839


In [5]:
df_regr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   anzahl_personen       216 non-null    float64
 1   vee                   399 non-null    int64  
 2   anzahl_kfz_je_person  216 non-null    float64
 3   unfaelle_je_10k_kfz   399 non-null    float64
 4   elektro               399 non-null    float64
 5   pih                   399 non-null    float64
 6   euro2                 399 non-null    float64
 7   euro3                 399 non-null    float64
 8   euro4                 399 non-null    float64
 9   euro6                 399 non-null    float64
 10  euro6dt               399 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 34.4 KB


## Variablen Listen

In [6]:
y_label = "euro4"
features = ["vee", "unfaelle_je_10k_kfz", "elektro", "pih", "euro2", "euro3", "euro6dt"]

X = df_regr[features]
y = df_regr[y_label]

y_label = "euro6dt"
features = ["vee", "unfaelle_je_10k_kfz", "elektro", "pih", "euro2", "euro3", "euro4"]

X = df_regr[features]
y = df_regr[y_label]

## Daten aufteilen, Train-Test-Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,     # 20% Testdaten
    random_state=42    # Für Reproduzierbarkeit
)

## Modell

In [8]:
def plot_regression_and_residuals(X, y):
    plots = []
    for feature in X.columns:
        # Berechne Steigung und Achsenabschnitt für einfache lineare Regression
        X_feature = X[[feature]]
        slope = np.cov(X_feature[feature], y)[0,1] / np.var(X_feature[feature])
        intercept = y.mean() - slope * X_feature[feature].mean()
        y_pred = intercept + slope * X_feature[feature]
        
        # Erstelle DataFrame für beide Plots
        df_plot = pd.DataFrame({
            feature: X_feature[feature],
            'y': y,
            'y_pred': y_pred,
            'residuals': y - y_pred
        })
        
        # Scatterplot mit Regressionsgerade
        base_scatter = alt.Chart(df_plot).encode(
            x=alt.X(feature, title=None)
        )
        
        scatter = base_scatter.mark_point().encode(
            y=alt.Y('y', title='y')
        ) + base_scatter.mark_line(color='red').encode(
            y=alt.Y('y_pred', title='y_pred')
        ).properties(
            height=200
        )
        
        # Geradengleichung als Text - mit verbesserter Darstellung für negative Steigung
        equation_text = f'y = {intercept:.2f} {"+" if slope >= 0 else "-"} {abs(slope):.2f} * x'
        text = alt.Chart(pd.DataFrame({
            feature: [X_feature[feature].min() + (X_feature[feature].max() - X_feature[feature].min()) * 0.1],
            'y': [y.max() - (y.max() - y.min()) * 0.1],
            'text': [equation_text]
        })).mark_text(align='left', baseline='top').encode(
            x=feature,
            y='y',
            text='text'
        )
        
        scatter = scatter + text
        
        # Residuenplot
        base_residual = alt.Chart(df_plot).encode(
            x=alt.X(feature, title=feature)
        )
        
        residual_plot = base_residual.mark_point().encode(
            y=alt.Y('residuals', title='Residuen')
        ).properties(
            height=100
        )
        
        # Kombiniere Plots vertikal mit gemeinsamer x-Achse
        combined_plot = alt.vconcat(scatter, residual_plot).resolve_scale(
            x='shared'
        )
        plots.append(combined_plot)
    
    # Anordnung der Plots in einem Raster mit 3 Spalten
    rows = []
    for i in range(0, len(plots), 3):
        row = alt.hconcat(*plots[i:i+3])
        rows.append(row)
    
    return alt.vconcat(*rows)

In [9]:
plot_regression_and_residuals(X, y)

alt.VConcatChart(...)

In [10]:
plot_regression_and_residuals(X[['elektro']], y)

alt.VConcatChart(...)

In [11]:
filtered_X = X[X['elektro'] <= 2]
filtered_y = y[filtered_X.index]

plot_regression_and_residuals(filtered_X[['elektro']], filtered_y)

alt.VConcatChart(...)

### Auswahl des Modells

In [12]:
regr = LinearRegression()

### Modell mit den Daten trainieren

In [13]:
regr.fit(X_train, y_train)

LinearRegression()

In [14]:
# Funktion zum Erstellen von Scatterplot mit Regressionsgerade und Residuenplot
def plot_regression_and_residuals(X_train, y_train, model):
    plots = []
    for i, feature in enumerate(X_train.columns):
        # Vorhersagen für die Regressionsgerade
        X_feature = X_train[[feature]]
        y_pred = model.intercept_ + model.coef_[i] * X_feature[feature]
        
        # Scatterplot mit Regressionsgerade
        scatter = alt.Chart(pd.DataFrame({
            feature: X_train[feature],
            'y_train': y_train,
            'y_pred': y_pred
        })).mark_point().encode(
            x=alt.X(feature, title=feature),
            y=alt.Y('y_train', title='y_train')
        ) + alt.Chart(pd.DataFrame({
            feature: X_train[feature],
            'y_pred': y_pred
        })).mark_line(color='red').encode(
            x=alt.X(feature, title=feature),
            y=alt.Y('y_pred', title='y_pred')
        ).properties(
            title=f'Scatterplot with Regression Line for {feature}'
        )
        
        # Residuen berechnen
        residuals = y_train - y_pred
        
        # Residuenplot
        residual_plot = alt.Chart(pd.DataFrame({
            feature: X_train[feature],
            'residuals': residuals
        })).mark_point().encode(
            x=alt.X(feature, title=feature),
            y=alt.Y('residuals', title='Residuals')
        ).properties(
            title=f'Residual Plot for {feature}'
        )
        
        # Kombiniere Scatterplot und Residuenplot
        combined_plot = alt.vconcat(scatter, residual_plot)
        plots.append(combined_plot)
    
    return alt.vconcat(*plots)
plot_regression_and_residuals(X_train, y_train, regr)

alt.VConcatChart(...)

In [15]:
# Bestimmtheitsmaß R² für Trainings- und Test Daten berechnen
r2_train = regr.score(X_train, y_train)
r2_test = regr.score(X_test, y_test)

print(f'R² Training: {r2_train:.4f}')
print(f'R² Test: {r2_test:.4f}')

R² Training: 0.8823
R² Test: 0.8002


**R² = 0.7764 auf Testdaten bedeutet:**

1. **Interpretation des R²-Werts**:
   - 77,6% der Varianz in den Testdaten wird durch das Modell erklärt
   - Ein Wert von 1.0 wäre perfekte Vorhersage
   - Ein Wert von 0.0 bedeutet keine Vorhersagekraft

2. **Bewertung**:
   - Guter Wert für reale Daten
   - Zeigt starken Zusammenhang zwischen Features und Zielvariable
   - Modell hat gute Generalisierungsfähigkeit, da auf Testdaten gemessen

3. **Einschränkungen**:
   - Etwa 22,4% der Varianz bleiben unerklärt
   - Weitere Faktoren könnten Einfluss haben
   - Mögliche nicht-lineare Zusammenhänge werden nicht erfasst
4. **Vergleich R² vs. adjustiertes R²**:
   - Reguläres R² ist typischerweise höher als adjustiertes R²
   - **Aber**: R² steigt automatisch mit der Anzahl der Features und bedeutet nicht unbedingt ein besseres Modell, da es durch zusätzliche Features künstlich aufgebläht sein könnte.
   - Adjustiertes R² berücksichtigt die Anzahl der Features und bestraft Overfitting. Daher ist adjustiertes R² aussagekräftiger für die Modellbewertung

In [16]:
def calculate_adjusted_r2(X: pd.DataFrame, 
                        y: pd.Series, 
                        model: LinearRegression = None) -> float:
    """
    Berechnet das angepasste R² für ein lineares Regressionsmodell.
    
    Args:
        X (pd.DataFrame): Features/Prädiktoren Matrix
        y (pd.Series): Zielvariable
        model (LinearRegression, optional): Vortrainiertes lineares Regressionsmodell.
            Falls None, wird ein neues Modell erstellt und trainiert.
    
    Returns:
        float: Angepasstes Bestimmtheitsmaß (R²)
            - Wertebereich: (-∞, 1]
            - 1: perfekte Vorhersage
            - 0: Modell ist nicht besser als der Mittelwert
            - < 0: Modell ist schlechter als der Mittelwert
            - Berücksichtigt die Anzahl der Features (p) und Beobachtungen (n)
            - Formel: 1 - (1 - R²) * (n-1)/(n-p-1)
    """
    if model is None:
        model = LinearRegression()
        model.fit(X, y)
    n, p = X.shape  # n: Anzahl der Beobachtungen, p: Anzahl der Features
    r2 = model.score(X, y)  # Bestimmtheitsmaß R²
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

In [17]:
print(f'Angepasstes R² Training: {calculate_adjusted_r2(X_train, y_train):.4f}')
print(f'Angepasstes R² Test: {calculate_adjusted_r2(X_test, y_test, model=regr):.4f}')

Angepasstes R² Training: 0.8797
Angepasstes R² Test: 0.7808


Mit Hilfe der Funktion `backward_elimination()` soll eine Rückwärtselimination durchgeführt werden: Es wird geprüft, ob das Entfernen eines Prädiktors das adjustierte 𝑅² verbessern würde. Dazu wird jeweils ein anderer Prädiktor ausgeschlossen und das adjustierte 𝑅² notiert. Das höchste adjustierte 𝑅², welches höher als das adjustierte 𝑅² des vollständigen Modells sein muss, beschreibt das neue Basismodell. Diese Schritte werden durchlaufen und erneut überprüft, bis das höchste adjustierte 𝑅² kleiner oder gleich dem adjustierten 𝑅² des Basismodells ist (ein weiterer Ausschluss eines der verbleibenden Prädiktoren würde zur Verschlechterung des Modells führen).

In [18]:
def backward_elimination(X: pd.DataFrame, 
                       y: pd.Series, 
                       verbose: bool = True) -> Tuple[List[str], float, List[Dict]]:
    """
    Führt eine Rückwärtselimination basierend auf dem adjustierten R² durch.
    
    Args:
        X (pd.DataFrame): Feature-Matrix
        y (pd.Series): Zielvariable
        verbose (bool): Wenn True, werden Zwischenergebnisse ausgegeben
    
    Returns:
        Tuple[List[str], float, List[Dict]]: 
            - Liste der besten Features
            - Finales adjustiertes R²
            - Historie der Elimination
    """
    features = list(X.columns)
    elimination_history = []
    
    # Initiales Modell
    best_adj_r2 = calculate_adjusted_r2(X, y)
    best_features = features.copy()
    
    if verbose:
        print(f"Start mit {len(features)} Features, Adj. R² = {best_adj_r2:.4f}")
    
    while len(features) > 1:
        results = {}
        # Evaluiere alle möglichen Feature-Kombinationen parallel
        for feature in features:
            remaining_features = [f for f in features if f != feature]
            X_subset = X[remaining_features]
            results[feature] = calculate_adjusted_r2(X_subset, y)
        
        # Finde bestes Ergebnis
        best_feature = max(results.items(), key=lambda x: x[1])
        
        # Prüfe ob Verbesserung
        if best_feature[1] > best_adj_r2:
            removed_feature = best_feature[0]
            best_adj_r2 = best_feature[1]
            features.remove(removed_feature)
            best_features = features.copy()
            
            elimination_history.append({
                'step': len(elimination_history) + 1,
                'removed_feature': removed_feature,
                'adj_r2': best_adj_r2,
                'n_features': len(features)
            })
            
            if verbose:
                print(f"Feature '{removed_feature}' entfernt: Adj. R² = {best_adj_r2:.4f}")
        else:
            break
            
    return best_features, best_adj_r2, elimination_history

In [19]:
# Rückwärtselimination durchführen
best_features, final_adj_r2, history = backward_elimination(X_train, y_train)

Start mit 7 Features, Adj. R² = 0.8797


In [20]:
# Ergebnisse ausgeben
print(f"Bestes adjustiertes R²: {final_adj_r2:.4f}")
print("\nSelektierte Features:")
for feature in best_features:
    print(f"- {feature}")

print("\nEliminations-Historie:")
for step in history:
    print(f"\nEntferntes Feature: {step['removed_feature']}")
    print(f"Adjustiertes R²: {step['adj_r2']:.4f}")

Bestes adjustiertes R²: 0.8797

Selektierte Features:
- vee
- unfaelle_je_10k_kfz
- elektro
- pih
- euro2
- euro3
- euro6dt

Eliminations-Historie:


In [21]:
# Modell trainieren mit den besten Prädiktoren
regr.fit(X_train[best_features], y_train)

LinearRegression()

In [22]:
# Berechne und zeige die Modellgüte mittels adjustierten R²
print(f'Angepasstes R² Training: {calculate_adjusted_r2(X_train, y_train, model=regr):.4f}')
print(f'Angepasstes R² Test: {calculate_adjusted_r2(X_test, y_test, model=regr):.4f}')

Angepasstes R² Training: 0.8797
Angepasstes R² Test: 0.7808
